In [1]:
using JuMP, Gurobi
import GLPK
import Random
import Plots

In [48]:
function generate_distance_matrix(n; random_seed = 1)
    rng = Random.MersenneTwister(random_seed)
    X = 100 * rand(rng, n)
    Y = 100 * rand(rng, n)
    d = [sqrt((X[i] - X[j])^2 + (Y[i] - Y[j])^2) for i in 1:n, j in 1:n]
    return X, Y, d
end

n = 4
X, Y, d = generate_distance_matrix(n)

([23.603334566204694, 34.651701419196044, 31.27069683360675, 0.790928339056074], [48.86128300795012, 21.096820215853597, 95.1916339835734, 99.99046588986135], [0.0 29.881964531335456 46.960513899062235 55.987491639154314; 29.881964531335456 0.0 74.17191260363222 85.85312621751531; 46.960513899062235 74.17191260363222 0.0 30.855227679377304; 55.987491639154314 85.85312621751531 30.855227679377304 0.0])

In [166]:
d = [
    0 10 15 20;
    10 0 25 25;
    15 25 0 30;
    20 25 30 0;
]
d =[
    0  10 15 20;
    10 0  35 25;
    15 35 0  30;
    20 25 30 0
]

4×4 Matrix{Int64}:
  0  10  15  20
 10   0  35  25
 15  35   0  30
 20  25  30   0

In [167]:
memo = fill(Inf, n, 1 << n)
memo

4×16 Matrix{Float64}:
 Inf  Inf  Inf  Inf  Inf  Inf  Inf  Inf  …  Inf  Inf  Inf  Inf  Inf  Inf  Inf
 Inf  Inf  Inf  Inf  Inf  Inf  Inf  Inf     Inf  Inf  Inf  Inf  Inf  Inf  Inf
 Inf  Inf  Inf  Inf  Inf  Inf  Inf  Inf     Inf  Inf  Inf  Inf  Inf  Inf  Inf
 Inf  Inf  Inf  Inf  Inf  Inf  Inf  Inf     Inf  Inf  Inf  Inf  Inf  Inf  Inf

In [181]:
function TSP_DP(i, mask)
    #base case
    d[1, 1] = 0
    if mask == ((1 << i) | 1) 
        return d[1,i]
    end
    #memoization
    if memo[i, mask] != -1
        return memo[i, mask]
    end

    res = 10^9
    for j in 1:n
        if (mask & (1<<j-1)) != 0 && j != i && j != 1
            res = min(res, TSP_DP(j, mask & (~(1 << i-1))) + d[j, i])
        end
    end
    memo[i, mask] = res
    return res
end

TSP_DP (generic function with 1 method)

In [182]:
ans = 10^9
for i in 1:n
    ans = min(ans, TSP_DP(i, (1 << n)-1) + d[i, 1])
end
print("The cost of most efficient tour = $ans")

The cost of most efficient tour = 1.0e9

In [190]:
function tsp_dynamic_programming(distance_matrix)
    n = size(distance_matrix, 1)
    num_states = 2^n
    dp = fill(Inf, n, num_states)

    # Initialize the base case
    dp[1, 1] = 0

    # Calculate the minimum distance for each state
    for mask in 1:num_states
        for u in 2:n
            if (mask & (1 << (u - 1))) != 0
                for v in 1:n
                    if v != u && (mask & (1 << (v - 1))) != 0
                        dp[u, mask] = min(dp[u, mask], dp[v, mask - (1 << (u - 1))] + distance_matrix[v, u])
                    end
                end
            end
        end
    end

    # Calculate the minimum tour length
    tour_length = Inf
    for u in 2:n
        tour_length = min(tour_length, dp[u, num_states - 1] + distance_matrix[u, 1])
        println(dp)
    end
    return tour_length
end 

tsp_dynamic_programming (generic function with 1 method)

In [191]:
tour_length = tsp_dynamic_programming(d)
println("Minimum tour length for cities:", tour_length)


[0.0 Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf; Inf Inf 10.0 Inf Inf Inf 50.0 Inf Inf Inf 45.0 Inf Inf Inf 70.0 Inf; Inf Inf Inf Inf 15.0 Inf 45.0 Inf Inf Inf Inf Inf 50.0 Inf 65.0 Inf; Inf Inf Inf Inf Inf Inf Inf Inf 20.0 Inf 35.0 Inf 45.0 Inf 75.0 Inf]
[0.0 Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf; Inf Inf 10.0 Inf Inf Inf 50.0 Inf Inf Inf 45.0 Inf Inf Inf 70.0 Inf; Inf Inf Inf Inf 15.0 Inf 45.0 Inf Inf Inf Inf Inf 50.0 Inf 65.0 Inf; Inf Inf Inf Inf Inf Inf Inf Inf 20.0 Inf 35.0 Inf 45.0 Inf 75.0 Inf]
[0.0 Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf Inf; Inf Inf 10.0 Inf Inf Inf 50.0 Inf Inf Inf 45.0 Inf Inf Inf 70.0 Inf; Inf Inf Inf Inf 15.0 Inf 45.0 Inf Inf Inf Inf Inf 50.0 Inf 65.0 Inf; Inf Inf Inf Inf Inf Inf Inf Inf 20.0 Inf 35.0 Inf 45.0 Inf 75.0 Inf]
Minimum tour length for cities:80.0


In [186]:
function tsp_dynamic_programming_memoization(distance_matrix)
    n = size(distance_matrix, 1)
    num_states = 2^n
    memo = Dict{Tuple{Int, Int}, Int}()

    function tsp(mask, pos)
        if haskey(memo, (mask, pos))
            return memo[(mask, pos)]
        end

        if mask == 2^n - 1
            return distance_matrix[pos, 1]
        end

        min_cost = Inf
        for next_city in 2:n
            if (mask & (1 << (next_city - 1))) == 0
                new_mask = mask | (1 << (next_city - 1))
                new_cost = distance_matrix[pos, next_city] + tsp(new_mask, next_city)
                min_cost = min(min_cost, new_cost)
            end
        end

        memo[(mask, pos)] = min_cost
        return min_cost
    end

    return tsp(1, 1)
end

# Example usage
distances = [
    0  10 15 20;
    10 0  35 25;
    15 35 0  30;
    20 25 30 0
]

result = tsp_dynamic_programming_memoization(distances)
println("Minimum tour length using memoization: $result")


Minimum tour length using memoization: 80.0
